# First Profiling

To develop a functional deconvolution pipeline, we need to optimize peak detection and windowing. All peaks need to be detected to fit each to the skew norm model, peak regions need to be seperated enough (either through sharpening or baseline subtraction) to deem them not convolved - seperated into different windows.

In [ ]:
%reload_ext autoreload
%autoreload 2
import sys
import pandera as pa
import hvplot
import hvplot.pandas  # noqa
import panel as pn
import polars as pl
from dataclasses import dataclass
from hplc_py.pipeline.preprocess_dashboard import DataSets, PreProcesser
from pandera.typing.polars import LazyFrame, DataFrame

pl.Config.set_tbl_rows(10)
pn.extension()
pn.config.comms = "vscode"
sys.executable

In [ ]:
dsets = DataSets()
ringland = dsets.ringland.fetch()
ringland.head()

In [ ]:
prepro = PreProcesser()
prepro.ingest_signal(ringland, time_col="time", amp_col="signal")

In [ ]:
prepro.signal_adjustment(bcorr__n_iter=39)
prepro.map_peaks(find_peaks_kwargs={"prominence": 0.01})
prepro.map_windows()
prepro.viz_preprocessing(opt_args={"height": 500, "width": 1000})

As we can see, an insufficient number of peaks have been correctly detected, and many peaks have shoulders which are obviously highly convolved peaks. We need to increase the resolution through sharpening to maximise the peak detection and definition prior to windowing.

The two factors for optimizing windowing are peak detection, peak width measurement, and basseline subtraction.

Peak detection can be incresed by decreasing the minimum prominence:

In [ ]:
prepro.signal_adjustment(bcorr__n_iter=39)
prepro.map_peaks(find_peaks_kwargs={"prominence": 0.0001})
prepro.map_windows()
prepro.viz_preprocessing(opt_args={"height": 1000, "width": 1500}).opts(
    height=750, width=1250
)

Much better. Have doubled the number of peaks detected. But there is still a shoulder on peak zero that isnt detected, another small one on peak two, 44, 45, etc. I think a default sharpening is necessary. Notes continue in [Smoothing and Sharpening](../pipeline/preprocesing_1_smoothing_sharpening.ipynb)